# Testing
### Eheschliessung nach Herkunft der Eheschliessenden, seit 1993
Datum: 02.11.2021

**Importiere die notwendigen Packages**

In [36]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt

import datetime
import geopandas as gpd
import folium 
import plotly.express as px
import seaborn as sns
import leafmap

import requests
import io


In [37]:
SSL_VERIFY = False
# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.gemeinderat-zuerich.ch nicht klappt (z.B. wegen Proxy)
# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)
# SSL_VERIFY = False

In [38]:
if not SSL_VERIFY:
    import urllib3
    urllib3.disable_warnings()

Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [39]:
#pd.options.display.float_format = lambda x : '{:,.1f}'.format(x) if (np.isnan(x) | np.isinf(x)) else '{:,.0f}'.format(x) if int(x) == x else '{:,.1f}'.format(x)
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

### Zeitvariabeln
Bestimme den aktuellst geladenen Monat. Hier ist es der Stand vor 2 Monaten. 
Bestimme noch weitere evt. sinnvolle Zeitvariabeln.

Zum Unterschied zwischen import datetime und from datedtime import datetime, siehe https://stackoverflow.com/questions/15707532/import-datetime-v-s-from-datetime-import-datetime

Zuerst die Zeitvariabeln als Strings

In [40]:
#today_date = datetime.date.today()
#date_time = datetime.datetime.strptime(date_time_string, '%Y-%m-%d %H:%M')
now = datetime.date.today()
date_today = now.strftime("%Y-%m-%d")
year_today = now.strftime("%Y")
month_today = now.strftime("%m")
day_today = now.strftime("%d")


Und hier noch die Zeitvariabeln als Integers:
- `aktuellesJahr`
- `aktuellerMonat`: Der gerade jetzt aktuelle Monat
- `selectedMonat`: Der aktuellste Monat in den Daten. In der Regel zwei Monate her.

In [41]:
#now = datetime.now() 
int_times = now.timetuple()

aktuellesJahr = int_times[0]
aktuellerMonat = int_times[1]
selectedMonat = int_times[1]-2

print(aktuellesJahr, 
      aktuellerMonat,
    'datenstand: ', selectedMonat,
     int_times)


2022 11 datenstand:  9 time.struct_time(tm_year=2022, tm_mon=11, tm_mday=3, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=307, tm_isdst=-1)


Berechne die Variable Epoche um später das SAS-Datum in ein Unix-Datum umzuwandeln. Bei SAS beginnt die Epoche am 1.1.1960. Bei Unix am 1.1.1970.
Diese Variable wird beim CSV-Import benötigt.

In [42]:
epoch = datetime.datetime(1960, 1, 1)

### Setze einige Pfadvariabeln

- Der Packagename ist eigentlich der **Verzeichnisname** unter dem die Daten und Metadaten auf der Dropzone abgelegt werden.
- Definiert wird er bei SASA-Prozessen auf dem **Produkte-Sharepoint ([Link](https://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/PersonalViews.aspx?PageView=Personal&ShowWebPart={6087A3E7-8AC8-40BA-8278-DECFACE124FF}))**.
- Der Packagename wird auf CKAN teil der URL, daher ist die exakte Schreibweise wichtig.

Beachte: im Packagename müssen alle Buchstaben **klein** geschrieben werden. Dies weil CKAN aus grossen kleine Buchstaben macht.

**BITTE HIER ANPASSEN**

In [43]:
package_name = "bev_ehe_herkunft_od3301"

In [44]:
dataset_name = "BEV330OD3301.csv"

**Statische Pfade in DWH-Dropzones**

In [45]:
dropzone_path_integ = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH"

In [46]:
dropzone_path_prod = r"\\szh\ssz\applikationen\OGD_Dropzone\DWH"

**Statische Pfade CKAN-URLs**

In [47]:
ckan_integ_url ="https://data.integ.stadt-zuerich.ch/dataset/int_dwh_"

In [48]:
ckan_prod_url ="https://data.stadt-zuerich.ch/dataset/"

### Checke die Metadaten auf der CKAN INTEG- oder PROD-Webseite

Offenbar lassen sich aktuell im Markdownteil keine Variabeln ausführen, daher gehen wir wie unten gezeigt vor. Siehe dazu: https://data-dive.com/jupyterlab-markdown-cells-include-variables
Instead of setting the cell to Markdown, create Markdown from withnin a code cell! We can just use python variable replacement syntax to make the text dynamic

In [49]:
from IPython.display import Markdown as md

In [50]:
md(" **1. Dataset auf INTEG-Datakatalog:** Link {} ".format(ckan_integ_url+package_name))

 **1. Dataset auf INTEG-Datakatalog:** Link https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_ehe_herkunft_od3301 

In [51]:
md(" **2. Dataset auf PROD-Datakatalog:** Link {} ".format(ckan_prod_url+package_name))

 **2. Dataset auf PROD-Datakatalog:** Link https://data.stadt-zuerich.ch/dataset/bev_ehe_herkunft_od3301 

### Importiere einen Datensatz 

Definiere zuerst folgende Werte:
1) Kommt der Datensatz von PROD oder INTEG?
2) Beziehst Du den Datensatz direkt ab der DROPZONE oder aus dem INTERNET?

In [52]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "integ"; #prod vs something else
data_source = "dropzone"; #dropzone vs something else
print(status+" - "+ data_source)

integ - dropzone


In [53]:
# Filepath
if status == "prod":
    if data_source == "dropzone":
            fp = dropzone_path_prod+"\\"+ package_name +"\\"+dataset_name
            print("fp lautet:"+fp)
    else:
        #fp = r"https://data.stadt-zuerich.ch/dataset/bau_neubau_whg_bausm_rinh_geb_projstatus_quartier_seit2009_od5011/download/BAU501OD5011.csv"
        fp = ckan_prod_url+package_name+'/download/'+dataset_name
        print("fp lautet:"+fp)
else:
    if data_source == "dropzone":
        fp = dropzone_path_integ+"\\"+ package_name +"\\"+dataset_name
        print("fp lautet:"+fp)
    else:
        #fp = r"https://data.stadt-zuerich.ch/dataset/bau_neubau_whg_bausm_rinh_geb_projstatus_quartier_seit2009_od5011/download/BAU501OD5011.csv"
        fp = ckan_integ_url+package_name+'/download/'+dataset_name
        print("fp lautet:"+fp)


fp lautet:\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bev_ehe_herkunft_od3301\BEV330OD3301.csv


Beachte, wie das SAS Datum (ohne Format) in ein UNIX Datum umgerechnet und als Datumsformat dargestellt wird! Siehe dazu `https://stackoverflow.com/questions/26923564/convert-sas-numeric-to-python-datetime`

In [76]:
# Read the data
if data_source == "dropzone":
    data2betested = pd.read_csv(
        fp
        , sep=','
        ,parse_dates=['StichtagDatJahr']
        ,low_memory=False
    )
    print("dropzone")
else:
    r = requests.get(fp, verify=False)  
    r.encoding = 'utf-8'
    data2betested = pd.read_csv(
        io.StringIO(r.text)
        ,parse_dates=['StichtagDatJahr']
        # KONVERTIERE DAS SAS DATUM IN EIN UNIXDATUM UND FORMATIERE ES
        #, date_parser=lambda s: epoch + datetime.timedelta(days=int(s))
        ,low_memory=False)
    print("web")

data2betested.dtypes

dropzone


StichtagDatJahr    datetime64[ns]
HerkunftEp1Lang            object
HerkunftEp1Sort             int64
HerkunftEp1Cd               int64
HerkunftEp2Lang            object
HerkunftEp2Sort             int64
HerkunftEp2Cd               int64
SexEp1Cd                    int64
SexEp1Lang                 object
SexEp1Kurz                 object
SexEp2Cd                    int64
SexEp2Lang                 object
SexEp2Kurz                 object
AnzSchl                     int64
dtype: object

Berechne weitere Attribute falls notwendig

In [77]:
data2betested = (
    data2betested
    .copy()
    .assign(
        #Aktualisierungs_Datum_str= lambda x: x.Aktualisierungs_Datum.astype(str),
        StichtagDatJahr_str = lambda x: x.StichtagDatJahr.astype(str),
    )
    .sort_values('StichtagDatJahr', ascending=False)
    )
data2betested.dtypes

StichtagDatJahr        datetime64[ns]
HerkunftEp1Lang                object
HerkunftEp1Sort                 int64
HerkunftEp1Cd                   int64
HerkunftEp2Lang                object
HerkunftEp2Sort                 int64
HerkunftEp2Cd                   int64
SexEp1Cd                        int64
SexEp1Lang                     object
SexEp1Kurz                     object
SexEp2Cd                        int64
SexEp2Lang                     object
SexEp2Kurz                     object
AnzSchl                         int64
StichtagDatJahr_str            object
dtype: object

### Einfache Datentests

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [78]:
#data2betested.head(6)

In [79]:
data2betested.dtypes

StichtagDatJahr        datetime64[ns]
HerkunftEp1Lang                object
HerkunftEp1Sort                 int64
HerkunftEp1Cd                   int64
HerkunftEp2Lang                object
HerkunftEp2Sort                 int64
HerkunftEp2Cd                   int64
SexEp1Cd                        int64
SexEp1Lang                     object
SexEp1Kurz                     object
SexEp2Cd                        int64
SexEp2Lang                     object
SexEp2Kurz                     object
AnzSchl                         int64
StichtagDatJahr_str            object
dtype: object

In [80]:
data2betested.shape

(116, 15)

Beschreibe einzelne Attribute

In [81]:
data2betested.describe()

,HerkunftEp1Sort,HerkunftEp1Cd,HerkunftEp2Sort,HerkunftEp2Cd,SexEp1Cd,SexEp2Cd,AnzSchl
count,116,116,116,116,116,116,116
mean,2,2,2,2,2,1,845
std,1,1,1,1,0,0,226
min,1,1,1,1,2,1,426
25%,1,1,1,1,2,1,650
50%,2,2,2,2,2,1,851
75%,2,2,2,2,2,1,1033
max,2,2,2,2,2,1,1363


Wie viele Nullwerte gibt es im Datensatz?

In [82]:
data2betested.isnull().sum()

StichtagDatJahr        0
HerkunftEp1Lang        0
HerkunftEp1Sort        0
HerkunftEp1Cd          0
HerkunftEp2Lang        0
HerkunftEp2Sort        0
HerkunftEp2Cd          0
SexEp1Cd               0
SexEp1Lang             0
SexEp1Kurz             0
SexEp2Cd               0
SexEp2Lang             0
SexEp2Kurz             0
AnzSchl                0
StichtagDatJahr_str    0
dtype: int64

Welches sind die Quartiere ohne Werte bei AnzBestWir?

In [83]:
data2betested[np.isnan(data2betested.AnzSchl)]

,StichtagDatJahr,HerkunftEp1Lang,HerkunftEp1Sort,HerkunftEp1Cd,HerkunftEp2Lang,HerkunftEp2Sort,HerkunftEp2Cd,SexEp1Cd,SexEp1Lang,SexEp1Kurz,SexEp2Cd,SexEp2Lang,SexEp2Kurz,AnzSchl,StichtagDatJahr_str


### Setze den Zeitindex

In [88]:
data2betested.set_index('StichtagDatJahr')

,HerkunftEp1Lang,HerkunftEp1Sort,HerkunftEp1Cd,HerkunftEp2Lang,HerkunftEp2Sort,HerkunftEp2Cd,SexEp1Cd,SexEp1Lang,SexEp1Kurz,SexEp2Cd,SexEp2Lang,SexEp2Kurz,AnzSchl,StichtagDatJahr_str
StichtagDatJahr,,,,,,,,,,,,,,
2021-01-01,Schweizer/in,1,1,Schweizer/in,1,1,2,weiblich,W,1,männlich,M,926,2021-01-01
2021-01-01,Schweizer/in,1,1,Ausländer/in,2,2,2,weiblich,W,1,männlich,M,426,2021-01-01
2021-01-01,Ausländer/in,2,2,Schweizer/in,1,1,2,weiblich,W,1,männlich,M,543,2021-01-01
2021-01-01,Ausländer/in,2,2,Ausländer/in,2,2,2,weiblich,W,1,männlich,M,941,2021-01-01
2020-01-01,Schweizer/in,1,1,Schweizer/in,1,1,2,weiblich,W,1,männlich,M,923,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994-01-01,Ausländer/in,2,2,Ausländer/in,2,2,2,weiblich,W,1,männlich,M,1048,1994-01-01
1993-01-01,Schweizer/in,1,1,Ausländer/in,2,2,2,weiblich,W,1,männlich,M,767,1993-01-01
1993-01-01,Ausländer/in,2,2,Schweizer/in,1,1,2,weiblich,W,1,männlich,M,615,1993-01-01


### Einfache Visualisierungen zur Plausi

Exploriere die Daten mit Pivottable.JS

In [89]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

**Comment:**

Seit 2022 kommt die Kombination der Möglichkeiten bei Ehepartnern nicht mehr vor. Dies muss selber gemacht werden und **fehlt hier noch**:

In [93]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]
#mySelection[[mySelection.NationFHistLang =='Schweiz']]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['HerkunftKomb'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(data2betested).encode(
    x='StichtagDatJahr',
    y='AnzSchl',
    color='HerkunftEp1Lang'
)

points = base.mark_circle().encode(
    opacity=alt.value(2)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

alt.LayerChart(...)

In [96]:
alt.Chart(data2betested).mark_circle(size=60).encode(
    x='StichtagDatJahr',
    y='AnzSchl',
    #color='AnzNat',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='greens')),
    color=alt.Color('HerkunftEp1Lang', scale=alt.Scale(scheme='dark2')),
    tooltip=['AnzSchl','StichtagDatJahr','HerkunftEp1Lang','HerkunftEp1Sort']
).interactive() # this makes the axes interactive: now you can zoom & pan




alt.Chart(...)

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](https://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/DispForm.aspx?ID=198&ContentTypeId=0x0100988EAF029F1EFE4CA675F53C32A5D53D01006DBC563E6FBE9E4EB6FDC780799752E1)**